# 문장 2207건 ChatGPT 넣어보기

In [21]:
import pandas as pd
from tqdm import tqdm
import os, sys
sys.path.append(os.path.abspath(".."))
from Read_Data import ReadKiprisData
from transformers import ElectraModel, ElectraTokenizer
from typing import Dict, Tuple
import re

In [6]:
# path 설정
patent_data_path = "../data/ner_dic_ami"
save_path = "../data/output"

In [98]:
read_data = ReadKiprisData()
data_df = read_data.run(patent_data_path)

100%|██████████| 4/4 [00:00<00:00,  8.55it/s]


In [9]:
# 모델 경로 (KIPI-KorPatELECTRA 경로 설정)
model_path = "../KIPI-KorPatELECTRA/KorPatELECTRA/PT"

# 토크나이저 및 모델 불러오기
tokenizer = ElectraTokenizer.from_pretrained(model_path)
model = ElectraModel.from_pretrained(model_path)

Some weights of the model checkpoint at ../KIPI-KorPatELECTRA/KorPatELECTRA/PT were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [99]:
# 1. 데이터 전처리
# ex) "1."" -> "제 1항"
def text_replace(text: str)-> str:
    new_text = re.sub(r"(\d)\.", lambda match: f"제 {match.group(1)} 항.", text)
    return new_text

sen_df = pd.DataFrame({'sen': list(data_df['문장'].unique())})
sen_df['sen'] = sen_df['sen'].apply(lambda sen: text_replace(sen))
sen_df.head()

,sen
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ..."
1,"제 2 항. 단말기에 콘텐트 리스트를 전송하는 콘텐트 리스트 제공부와,분산처리 작업..."
2,"제 3 항. 제 2 항에 있어서, 상기 스트리밍 처리부는 상기 단말기로부터 선택된 ..."
3,"제 4 항. 제 3 항에 있어서, 상기 스트리밍 처리부는 상기 콘텐트 목록에 속한 ..."
4,"제 5 항. 제 4 항에 있어서, 상기 작업량 계산부는 상기 콘텐트의 메타 정보에 ..."


In [67]:
# 2. 확장장
def tokenizing_result(sen: str)-> Tuple[list, list]:
    tokens = tokenizer.tokenize(sen)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    input_ids = tokenizer(sen)['input_ids']

    return tokens, input_ids

tqdm.pandas(desc="문장 토크나이징")

sen_df[["tokens", "input_ids"]] = sen_df["sen"].progress_apply(
    lambda sentence: pd.Series(tokenizing_result(sentence)))

expanded_df = sen_df.explode(["tokens", "input_ids"]).reset_index(drop=True) # 리스트를 행으로 확장
expanded_df

문장 토크나이징: 100%|██████████| 2207/2207 [00:03<00:00, 637.31it/s]


,sen,tokens,input_ids
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",[CLS],2
1,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",제,1749
2,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",1,10
3,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",항,2390
4,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",.,8
...,...,...,...
141041,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,이동통신단말기,10854
141042,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##의,2715
141043,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,인증,5276
141044,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##시스템,5671


In [ ]:
# expanded_df.to_csv(os.path.join(save_path, "patentData_token_ids_2207.csv"), encoding='euc-kr')

In [90]:
def tokenizing_result(sen: str)-> Tuple[list, list]:
    tokens = tokenizer.tokenize(sen)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]

    return tokens

tqdm.pandas(desc="문장 토크나이징")

sen_df["tokens"] = sen_df["sen"].progress_apply(
    lambda sentence:tokenizing_result(sentence))

expanded_df2 = sen_df.explode("tokens").reset_index(drop=True) # 리스트를 행으로 확장
expanded_df2

문장 토크나이징: 100%|██████████| 2207/2207 [00:01<00:00, 1757.56it/s]


,sen,tokens
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",[CLS]
1,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",제
2,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",1
3,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",항
4,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",.
...,...,...
141041,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,이동통신단말기
141042,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##의
141043,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,인증
141044,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##시스템


In [93]:
# 문장 번호 추가 
expanded_df2['sen_idx'] = expanded_df2.groupby('sen').ngroup()
expanded_df2

,sen,tokens,sen_idx
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",[CLS],1402
1,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",제,1402
2,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",1,1402
3,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",항,1402
4,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",.,1402
...,...,...,...
141041,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,이동통신단말기,1293
141042,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##의,1293
141043,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,인증,1293
141044,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##시스템,1293


In [97]:
# expanded_df2.to_csv("patentData_token_idx_2207.csv", encoding='euc-kr')

In [ ]:
sen_df